## 먼저 소요리문답자료를 크롤링합니다.

### 소요리문답 목록 페이지에서 문답별 파라미터 수집(POST방식 Query)
1. 다음을 반복한다.
    1. 소요리문답 목록 페이지에 접근한다.
    1. 소요리문답 목록 없음 페이지면 반복을 중지한다.
    1. 개별 소요리문답의 POST방식 Query 파라미터를 추출한다.
    1. 페이지 번호를 1 증가한다.

### 개별 게시물 수집
1. 다음을 반복한다.
    1. 개별 소요리문답의 POST방식 Query 파라미터를 이용하여 개별 소요리문답 페이지에 접근한다.
    1. 소요리문답 번호을 추출한다.
    1. 소요리문답 질문을 추출한다.
    1. 소요리문답 답문을 추출한다.
    1. 다음 POST방식 Query 파라미터로 넘어간다.
    
### JSON파일로 저장한다.

In [1]:
import requests
from bs4 import BeautifulSoup
import ujson
import time
import re

"""
소요리문답 목록 페이지에서 문답별 파라미터 수집(POST방식 Query)
1. 다음을 반복한다.
    1. 소요리문답 목록 페이지에 접근한다.
    2. 소요리문답 목록 없음 페이지면 반복을 중지한다.
    3. 개별 소요리문답의 POST방식 Query 파라미터를 추출한다.
    4. 페이지 번호를 1 증가한다.

개별 소요리문답 수집
2. 다음을 반복한다.
    1. 개별 소요리문답의 POST방식 Query 파라미터를 이용하여 개별 소요리문답 페이지에 접근한다.
    2. 소요리문답 번호을 추출한다.
    3. 소요리문답 질문을 추출한다.
    4. 소요리문답 답문을 추출한다.
    5. 다음 POST방식 Query 파라미터로 넘어간다.
"""

# 소요리문답 목록 페이지에서 문답별 파라미터 수집(POST방식 Query)
def collect_post_urls():
    """목록 페이지에서 개별 소요리문답의 POST방식 Query 파라미터를 수집하여 돌려준다."""
    
    all_post_urls = []
    page_num = 1
    
    while True:
        print("소요리문답의 목록 페이지 {}에 접근합니다.".format(page_num))
        list_page = get_list_page(page_num)
        
        if is_empty_page(list_page):
            break
            
        post_urls = extract_post_urls(list_page)
        for post_url in post_urls:
            all_post_urls.append(post_url)
        page_num += 1
        
    print("총 {}개의 소요리문답의 POST방식 Query 파라미터가 수집되었습니다.".format(len(all_post_urls)))
        
    return all_post_urls

        
def get_list_page(page_num):
    """주어진 페이지 번호에 해당하는 소요리문답 목록 페이지에 접근하여 soup 객체를 만들어 돌려준다."""
    
    list_page_url = gen_list_page_url()
    post_param = {'pno': "{}".format(page_num)} 
    soup = get_soup(list_page_url,post_param)
    
    return soup


def get_soup(url,post_params):   
    """주어진 POST방식 Query 파라미터로 접속하여 HTML을 얻어서 soup 객체를 만들어 돌려준다."""
    
    #resp = requests.get(url)
    resp = requests.post(url, data=post_params)
    resp.encoding=None   # None 으로 설정
    html = resp.text
    soup = BeautifulSoup(html, "lxml")
    time.sleep(1.5)    
    
    return soup


def gen_list_page_url():
    """소요리문답 목록 페이지 URL을 돌려준다."""
    
    # 목표 url : 대한예수교장로회총회->총회헌법->소요리문답
    list_page_url = "http://www.gapck.org/sub_06/sub03_01.asp"
    
    return list_page_url
    
    
def is_empty_page(list_page):
    """주어진 페이지가 빈 페이지인지 여부를 판단한다."""
    
    if "허용되지 않은 접근입니다" in list_page.get_text():
        return True
    
    return False

    
def extract_post_urls(list_page):
    """주어진 페이지에서 개별 소요리문답의 POST방식 Query 파라미터를 추출한다."""

    post_url_elems = list_page.find_all("a", {"class": "b_color_03"})
    
    post_urls = []
    for post_url_elem in post_url_elems:
        post_url = re.findall("(\d+)",post_url_elem.attrs['href'])
        post_urls.append(post_url)
    
    
    return post_urls

# 개별 소요리문답 수집
def collect_posts(output_file_name, all_post_urls):
    """주어진 소요리문답 POST방식 Query 파라미터들에 접근하여 소요리문답을 수집하여 출력 파일에 저장한다."""
    
    # 목표 개별 소요리문답 url
    url = "http://www.gapck.org/sub_06/sub03_02.asp"

    num_post_urls = len(all_post_urls)
    
    with open(output_file_name, "w", encoding="utf-8") as output_file:
        for i, post_url in enumerate(all_post_urls, 1):
            #print("소요리문답 POST방식 Query 파라미터 {}에 접근합니다. ({}/{})".format(post_url, i, num_post_urls))
            
            #params = ['18', '90', '2', '1']
            qnaNo = post_url[0]
            reversQnaNo = post_url[1]
            pageNo = post_url[2]

            post_param = {'hno': "{}".format(qnaNo), 'rno': "{}".format(reversQnaNo), 'pno': "{}".format(pageNo)} 
            
            post_page = get_soup(url,post_param)
            catechism_no = extract_no(post_page)
            catechism_question = extract_question(post_page)
            catechism_answer = extract_answer(post_page)
            write_post(output_file, catechism_no, catechism_question, catechism_answer)
            # break

            
def extract_no(post_page):
    """주어진 soup 객체에서 소요리문답 번호을 추출하여 돌려준다."""

    catechism_elem = post_page.find("td", {"class": "G0_654400"})
    catechism_no = catechism_elem.text
    
    return catechism_no
            
    
def extract_question(post_page):
    """주어진 soup 객체로부터 소요리문답 질문을 추출하여 돌려준다."""
    
    question_elem = post_page.find("a", {"class": "b_color_03"})
    question = question_elem.text.strip()
    
    return question 


def extract_answer(post_page):
    """주어진 soup 객체에서 게시물 본문을 추출하여 돌려준다."""
    
    # 본문의 CSS 선택자를 정규표현을 이용하여 지정한다.
    answer_elem = post_page.find("td", {"class": "color-title7_brown"})
    answer = answer_elem.text.strip()     
    
    return answer


def write_post(output_file, catechism_no, catechism_question, catechism_answer):
    """주어진 게시물 요소를 JSON 문자열로 만들어 출력 파일에 기록한다."""
    
    post = {"no": catechism_no, "question": catechism_question, "answer": catechism_answer}
    print(ujson.dumps(post, ensure_ascii=False), file=output_file)
    
    
def main():
    """소요리문답자료를 수집하여 저장한다."""
    
    output_file_name = "data/crawling/catechism.txt"
    all_post_urls = collect_post_urls()
    collect_posts(output_file_name, all_post_urls)
    print("총 {}개의 소요리문답 자료를 저장하였습니다.".format(len(all_post_urls)))
    
    
main()

소요리문답의 목록 페이지 1에 접근합니다.
소요리문답의 목록 페이지 2에 접근합니다.
소요리문답의 목록 페이지 3에 접근합니다.
소요리문답의 목록 페이지 4에 접근합니다.
소요리문답의 목록 페이지 5에 접근합니다.
소요리문답의 목록 페이지 6에 접근합니다.
소요리문답의 목록 페이지 7에 접근합니다.
소요리문답의 목록 페이지 8에 접근합니다.
소요리문답의 목록 페이지 9에 접근합니다.
소요리문답의 목록 페이지 10에 접근합니다.
소요리문답의 목록 페이지 11에 접근합니다.
소요리문답의 목록 페이지 12에 접근합니다.
총 107개의 소요리문답의 POST방식 Query 파라미터가 수집되었습니다.
총 107개의 소요리문답 자료를 저장하였습니다.


In [2]:
list_page = get_list_page("1")
print("list_page : [%s]" % list_page)

post_urls = extract_post_urls(list_page)
print("urls : [%s]" % post_urls)
for post_url in post_urls:
    all_post_urls.append(post_url)
    print("url : [%s]" % post_url)

list_page : [<html>
<head>
<title>대한예수교장로회총회</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="../text.css" rel="stylesheet" type="text/css"/>
<style type="text/css">
<!--
body {
	margin-left: 0px;
	margin-top: 0px;
}
-->
</style>
<script language="JavaScript" type="text/JavaScript">
<!--



function MM_preloadImages() { //v3.0
  var d=document; if(d.images){ if(!d.MM_p) d.MM_p=new Array();
    var i,j=d.MM_p.length,a=MM_preloadImages.arguments; for(i=0; i<a.length; i++)
    if (a[i].indexOf("#")!=0){ d.MM_p[j]=new Image; d.MM_p[j++].src=a[i];}}
}
//-->
</script>
<!--------------------------------- 스크립트 ---------------------------------->
<script language="javascript" src="js/common_script.js"></script>
<script language="javascript" src="js/common.js"></script>
<script language="JAVASCRIPT" type="TEXT/JAVASCRIPT">

	//---- 목록 묶음 이동을 위한 스크립트
	function gotopage(pagenum){
		thisfrm = document.L_form;
		thisfrm.pno.value = pagenum;
		thisfrm.KEY_TYPE.

NameError: name 'all_post_urls' is not defined